# Anàlisi de vendes per categoria i any

In [ ]:
# Importa pandas com a pd i numpy com a np. 
# Crea un DataFrame anomenat sales_df amb les següents dades:

#| category    | product    | year | units_sold | price_per_unit |
#| ----------- | ---------- | ---- | ---------- | -------------- |
#| Electronics | Laptop     | 2020 | 150        | 900            |
#| Electronics | Smartphone | 2021 | 300        | 700            |
#| Furniture   | Chair      | 2020 | 500        | 50             |
#| Furniture   | Table      | 2021 | 200        | 120            |
#| Clothing    | T-shirt    | 2020 | 800        | 20             |
#| Clothing    | Jacket     | 2021 | 400        | 80             |
#| Electronics | Tablet     | 2020 | 250        | 400            |
#| Furniture   | Sofa       | 2021 | 150        | 500            |
#| Clothing    | Shoes      | 2021 | 300        | 100            |


In [1]:
import pandas as pd
import numpy as np
data = {
    'category': ['Electronics', 'Electronics', 'Furniture', 'Furniture', 'Clothing', 
                 'Clothing', 'Electronics', 'Furniture', 'Clothing'],
    'product': ['Laptop', 'Smartphone', 'Chair', 'Table', 'T-shirt', 
                'Jacket', 'Tablet', 'Sofa', 'Shoes'],
    'year': [2020, 2021, 2020, 2021, 2020, 2021, 2020, 2021, 2021],
    'units_sold': [150, 300, 500, 200, 800, 400, 250, 150, 300],
    'price_per_unit': [900, 700, 50, 120, 20, 80, 400, 500, 100]
}
df = pd.DataFrame(data)
df

,category,product,year,units_sold,price_per_unit
0,Electronics,Laptop,2020,150,900
1,Electronics,Smartphone,2021,300,700
2,Furniture,Chair,2020,500,50
3,Furniture,Table,2021,200,120
4,Clothing,T-shirt,2020,800,20
5,Clothing,Jacket,2021,400,80
6,Electronics,Tablet,2020,250,400
7,Furniture,Sofa,2021,150,500
8,Clothing,Shoes,2021,300,100


In [ ]:
# Càlcul del total de vendes

# Afegeix una nova columna anomenada total_sales, que sigui el producte de units_sold i price_per_unit.



In [2]:
df['total_sales'] = df['units_sold'] * df['price_per_unit']
df

,category,product,year,units_sold,price_per_unit,total_sales
0,Electronics,Laptop,2020,150,900,135000
1,Electronics,Smartphone,2021,300,700,210000
2,Furniture,Chair,2020,500,50,25000
3,Furniture,Table,2021,200,120,24000
4,Clothing,T-shirt,2020,800,20,16000
5,Clothing,Jacket,2021,400,80,32000
6,Electronics,Tablet,2020,250,400,100000
7,Furniture,Sofa,2021,150,500,75000
8,Clothing,Shoes,2021,300,100,30000


In [ ]:
# Agregació bàsica

# Calcula la mitjana de vendes (total_sales) per categoria utilitzant groupby().



In [6]:
df.groupby('category')['total_sales'].mean()

category
Clothing        26000.000000
Electronics    148333.333333
Furniture       41333.333333
Name: total_sales, dtype: float64

In [ ]:
# Agregació múltiple

# Calcula, per cada categoria, el nombre total de productes, la mitjana de preu per unitat i la suma total de vendes.

In [3]:
# Agregació múltiple per categoria: nombre de productes, mitjana preu i suma total de vendes
agg = df.groupby('category').agg(
    total_products=('product', 'nunique'),
    avg_price_per_unit=('price_per_unit', 'mean'),
    total_sales=('total_sales', 'sum')
).reset_index()

agg

,category,total_products,avg_price_per_unit,total_sales
0,Clothing,3,66.666667,78000
1,Electronics,3,666.666667,445000
2,Furniture,3,223.333333,124000


In [ ]:
# Agrupació per múltiples columnes

# Agrupa les dades per category i year, i mostra el total de vendes per cada combinació.

In [4]:
# Agrupa per category i year i mostra el total de vendes per combinació
sales_by_cat_year = df.groupby(['category','year'])['total_sales'].sum().reset_index(name='total_sales')

sales_by_cat_year

,category,year,total_sales
0,Clothing,2020,16000
1,Clothing,2021,62000
2,Electronics,2020,235000
3,Electronics,2021,210000
4,Furniture,2020,25000
5,Furniture,2021,99000


In [ ]:
# Filtratge de grups

# Filtra i mostra només les categories que tinguin una mitjana de vendes superior a 30.000.

In [5]:
# Filtra categories amb mitjana de vendes superior a 30.000
mean_sales = df.groupby('category')['total_sales'].mean()

high_mean_categories = mean_sales[mean_sales > 30000]

high_mean_categories

category
Electronics    148333.333333
Furniture       41333.333333
Name: total_sales, dtype: float64

In [ ]:
# Transformació

# Resta la mitjana de vendes de cada categoria a les seves files corresponents (centrant les dades per categoria).

In [7]:
# Centra les vendes restant la mitjana de vendes de cada categoria

df['sales_centered'] = df.groupby('category')['total_sales'].transform(lambda x: x - x.mean())

df[['category','product','total_sales','sales_centered']]

,category,product,total_sales,sales_centered
0,Electronics,Laptop,135000,-13333.333333
1,Electronics,Smartphone,210000,61666.666667
2,Furniture,Chair,25000,-16333.333333
3,Furniture,Table,24000,-17333.333333
4,Clothing,T-shirt,16000,-10000.000000
5,Clothing,Jacket,32000,6000.000000
6,Electronics,Tablet,100000,-48333.333333
7,Furniture,Sofa,75000,33666.666667
8,Clothing,Shoes,30000,4000.000000


In [ ]:
# Aplicació de funció personalitzada

# Crea una funció anomenada normalize_sales(x) que divideixi les vendes totals de cada producte pel màxim de vendes del grup (categoria).
# Aplica aquesta funció a sales_df utilitzant groupby('category').apply().



In [9]:
# Normalitza vendes dins de cada categoria (dividir per el màxim del grup)

def normalize_sales(x):
    """Divideix les vendes totals del grup pel valor màxim del grup.
    Retorna zeros si el màxim és 0 per evitar divisió per zero."""
    mx = x.max()
    return x / mx if mx != 0 else x * 0

# Aplica amb groupby.transform per mantenir l'index alineat amb df
# transform retorna una Series amb el mateix index que el DataFrame original

df['normalized_sales'] = df.groupby('category')['total_sales'].transform(lambda x: x / x.max() if x.max() != 0 else 0)

df[['category','product','total_sales','normalized_sales']]

,category,product,total_sales,normalized_sales
0,Electronics,Laptop,135000,0.642857
1,Electronics,Smartphone,210000,1.000000
2,Furniture,Chair,25000,0.333333
3,Furniture,Table,24000,0.320000
4,Clothing,T-shirt,16000,0.500000
5,Clothing,Jacket,32000,1.000000
6,Electronics,Tablet,100000,0.476190
7,Furniture,Sofa,75000,1.000000
8,Clothing,Shoes,30000,0.937500
